# 项目：整理Netflix电影演员评分数据

## 分析目标

此数据分析的目的是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，各演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

本实战项目的目的在于练习整理数据，从而得到可供下一步分析的数据。

## 简介

原始数据集记录了截止至2022年7月美国地区可观看的所有Netflix电视剧及电影数据。数据集包含两个数据表：`titles.csv`和`credits.csv`。

`titles.csv`包含电影及电视剧相关信息，包括影视作品ID、标题、类型、描述、流派、IMDB（一个国外的在线评分网站）评分，等等。`credits.csv`包含超过7万名出现在Netflix影视作品的导演及演员信息，包括名字、影视作品ID、人物名、演职员类型（导演/演员）等。

`titles.csv`每列的含义如下：
- id：影视作品ID。
- title：影视作品标题。
- show_type：作品类型，电视节目或电影。
- description：简短描述。
- release_year：发布年份。
- age_certification：适龄认证。
- runtime：每集电视剧或电影的长度。
- genres：流派类型列表。
- production_countries：出品国家列表。
- seasons：如果是电视剧，则是季数。
- imdb_id：IMDB的ID。
- imdb_score：IMDB的评分。
- imdb_votes：IMDB的投票数。
- tmdb_popularity：TMDB的流行度。
- tmdb_score：TMDB的评分。

`credits.csv`每列的含义如下：
- person_ID：演职员ID。
- id：参与的影视作品ID。
- name：姓名。
- character_name：角色姓名。
- role：演职员类型，演员或导演。

## 读取数据

导入数据分析所需要的库，并通过Pandas的`read_csv`函数，将原始数据文件"titles.csv"里的数据内容，解析为DataFrame并赋值给变量`original_titles`。将原始数据文件"credits.csv"里的数据内容，解析为DataFrame并赋值给变量`original_credits`。

In [ ]:
import pandas as pd

In [ ]:
original_titles = pd.read_csv("titles.csv")
original_credits = pd.read_csv("credits.csv")

In [ ]:
original_titles.head()

In [ ]:
original_credits.head()

# 评估和清理数据

在这一部分中，我们将对在上一部分建立的`original_titles`及`original_credits`DataFrame所包含的数据进行评估和清理。

主要从两个方面进行：结构和内容，即整齐度和干净度。

数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

为了区分开经过清理的数据和原始的数据，我们创建新的变量`cleaned_titles`，让它为`original_titles`复制出的副本，以及创建新的变量`cleaned_credits`，让它为`original_credits`复制出的变量。我们之后的清理步骤都将被运用在`cleaned_titles`和`cleaned_credits`上。

In [ ]:
cleaned_titles = original_titles.copy()
cleaned_credits = original_credits.copy()

## 数据整齐度

In [ ]:
cleaned_titles.head(10)

从数据的部分10行来看，`cleaned_titles`里的`genres`和`production_countries`的变量中包含多个值，应当进行拆分。

先提取任意一个`genres`变量的值进行观察。

In [ ]:
cleaned_titles['genres'][1]

虽然`genres`表示形式是列表，但其实际类型并非字符串列表，而是字符串，无法直接用`value_counts`统计各个值出现的次数。
我们可以使用Python内置的`eval`函数，它可以把字符串转换成表达式，所以可以帮我们把表示列表的字符串转换成列表本身。

In [ ]:
cleaned_titles['genres'] = cleaned_titles['genres'].apply(lambda s: eval(s))
cleaned_titles['genres'][1]

转换为列表后，就能用DataFrame的`explode`方法，把那个列的列表值拆分成单独的行。

In [ ]:
cleaned_titles = cleaned_titles.explode("genres")
cleaned_titles.head(10)

接下来，针对`production_countries`列也是一样的流程。

每个观察值的`production_countries`值并不表示单个流派，而是一系列流派。先提取任意一个`production_countries`变量的值进行观察。

In [ ]:
cleaned_titles['production_countries'][1]

可以看到，`production_countries`也是一样的问题，虽然表示形式是列表，但其实际类型并非字符串列表，而是字符串，难以进行拆分。
我们可以再次利用`eval`函数进行类型转换，并检查转换后确实是列表类型。

In [ ]:
cleaned_titles['production_countries'] = cleaned_titles['production_countries'].apply(lambda s: eval(s))
cleaned_titles['production_countries'][0]

In [ ]:
cleaned_titles = cleaned_titles.explode('production_countries')
cleaned_titles.head(10)

在处理完`cleaned_titles`的结构性问题后，查看`cleaned_credits`。

In [ ]:
cleaned_credits.head(10)

从头部的10行数据来看，`cleaned_credits`数据符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”，因此不存在结构性问题。

## 数据干净度

接下来通过`info`，对数据内容进行大致了解。

In [ ]:
cleaned_titles.info()

从输出结果来看，`cleaned_titles`数据共有17818条观察值，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量均存在缺失值，将在后续进行评估和清理。

此外，`release_year`表示年份，数据类型不应为数字，应为日期，所以需要进行数据格式转换。

In [ ]:
cleaned_titles["release_year"] = pd.to_datetime(cleaned_titles["release_year"], format='%Y')
cleaned_titles["release_year"]

In [ ]:
cleaned_credits.info()

从输出结果来看，`cleaned_credits`数据共有77801条观察值，其中`character`变量存在缺失值，将在后续进行评估和清理。

此外，`person_id`表示演职员ID，数据类型不应为数字，应为字符串，所以需要进行数据格式转换。

In [1]:
cleaned_credits["person_id"] = cleaned_credits["person_id"].astype("str")
cleaned_credits["person_id"]

NameError: name 'cleaned_credits' is not defined

### 处理缺失数据

在`cleaned_titles`中，`title`、`description`、`age_certification`、`genres`、`production_countries`、`seasons`、`imdb_id`、`imdb_score`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量存在缺失值。

由于影视作品的标题、描述、适龄认证、发行国家、电视剧季数、IMDB的ID、TMDB的流行度、TMDB的评分，并不影响我们挖掘各个流派中的高IMDB评分作品演员，所以可以保留`title`、`description`、`age_certification`、`production_countries`、`seasons`、`imdb_id`、`tmdb_popularity`、`tmdb_score`、`imdb_votes`、`tmdb_popularity`、`tmdb_score`变量值存在空缺的观察值。

但`imdb_score`和`genres`，即IMDB评分和流派，和我们后续要做的分析息息相关。

先提取出`imdb_score`缺失观察值进行查看。

In [ ]:
cleaned_titles.query("imdb_score.isnull()")

由于缺失分析所需的核心数据`imdb_score`，我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [ ]:
cleaned_titles = cleaned_titles.dropna(subset=["imdb_score"])
cleaned_titles["imdb_score"].isnull().sum()

In [ ]:
cleaned_titles.query("genres.isnull()")

由于缺失分析所需的核心数据`genres`，我们将把这些观察值删除，并查看删除后该列空缺值个数和：

In [ ]:
cleaned_titles = cleaned_titles.dropna(subset=["genres"])
cleaned_titles["genres"].isnull().sum()

接下来评估`cleaned_credits`的缺失数据，其中只有`character`变量存在缺失值。

角色名并不影响我们挖掘各个流派中的高IMDB评分作品演员，并且此变量缺失也有可能因为演职员类别是导演，没有对应角色，因此可以保留`character`变量值存在空缺的观察值。

### 处理重复数据

根据数据变量的含义以及内容来看，`cleaned_titles`里不应该存在每个变量值都相同的观察值，因此查看是否存在重复值。

In [ ]:
cleaned_titles.duplicated().sum()

输出结果为0，说明不存在重复值。

接下来查看`cleaned_credits`数据表是否存在重复值。

In [ ]:
cleaned_credits.duplicated().sum()

输出结果为0，说明不存在重复值。

### 处理不一致数据

针对`cleaned_titles`，不一致数据可能存在于`genres`和`character`变量中，我们将查看是否存在多个不同值指代同一流派，以及多个不同值指代同一国家的情况。

In [ ]:
cleaned_titles['genres'].value_counts()

从上面看出，`genres`列里并不存在不一致数据，各个值都在指代不同的流派。但是里面还存在空字符串表示的流派，并非有效数据，因此可以进行删除。

删除后，查看`cleaned_titles`里是否还存在`genres`为空字符串的行：

In [ ]:
cleaned_titles = cleaned_titles.query('genres != ""')
cleaned_titles.query('genres == ""')

接下来，针对`production_countries`列也是一样的流程，利用`value_counts`方法，得到`production_countries`的列表里面各个值的出现次数。

In [ ]:
cleaned_titles['production_countries'].value_counts()

由于`value_counts`执行结果中有太多值，Pandas只会默认显示开头和结尾的一些值。要完整展示结果，可以把`display.max_rows`设置为`None`，即取消展示行数上限。

但因为我们只是在当前调用`value_counts`时才需要看完整结果，所以可以结合`option_context`，只更改临时上限。

In [ ]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles['production_countries'].value_counts())

从以上输出结果来看，出品国家都用两位的国家代码来表示，除了里面存在一个的`Lebanon`值。

`Lebanon`的国家代码是`LB`，出现了39次，说明此处数据不一致。`LB`和`Lebanon`都在表示同一国家，需要进行统一。

把`cleaned_titles`里，`production_countries`的`"LB"`和`"Lebanon"`统一为`LB`，并检查替换后是否还存在`"LB"`：

对每个观察值"production_countries"列的列表运用上面的函数

In [ ]:
cleaned_titles["production_countries"] = cleaned_titles["production_countries"].replace({"Lebanon": "LB"})

### 检查"Lebanon"是否还存在

In [ ]:
with pd.option_context('display.max_rows', None):
    print(cleaned_titles.explode('production_countries')['production_countries'].value_counts())

另外，里面还存在空字符串表示的国家代码，并非有效数据。但由于出品国家并非分析所需的关键信息，所以可以保留出品国家为空的观察值。

针对`original_credits`，不一致数据可能存在于`role`中，我们将查看是否存在多个不同值指代同一演职员类型的情况。

In [ ]:
original_credits['role'].value_counts()

从以上输出结果来看，`role`只有两种可能的值，`ACTOR`或`DIRECTOR`，不存在不一致数据。我们可以把这列的类型转换为`Category`，好处是比字符串类型更节约内存空间，也能表明说值的类型有限。

In [ ]:
cleaned_credits["role"] = cleaned_credits["role"].astype("category")
cleaned_credits["role"]

### 处理无效或错误数据

可以通过DataFrame的`describe`方法，对数值统计信息进行快速了解。

In [ ]:
original_titles.describe()

从以上统计信息来看，`original_titles`里不存在脱离现实意义的数值。

`original_credits`由于不包含表示数值含义的变量，因此无需用`describe`检查。

# 整理数据

In [ ]:
cleaned_titles

In [ ]:
cleaned_credits

对数据的整理，与分析方向紧密相关。此次数据分析目标是，整理不同流派影视作品，比如喜剧片、动作片、科幻片中，演员出演作品的平均IMDB评分，从而挖掘出各个流派中的高评分作品演员。

那为了能同时获得流派与演员数据，我们需要把`cleaned_titles`和`cleaned_credits`，通过`id`作为键进行连接，因为两个数据表中`id`都是影视作品ID。

In [ ]:
credits_with_titles = pd.merge(cleaned_credits, cleaned_titles, on="id", how="inner")

连接后，我们就能知道各个演职员参与过的影视作品的具体信息。

In [ ]:
credits_with_titles.head()

由于我们只对挖掘演员的参演作品口碑感兴趣，导演不在我们的分析范围内，因此根据`role`，筛选出类型为`ACTOR`的观察值，供后续分析。

In [ ]:
actor_with_titles = credits_with_titles.query('role == "ACTOR"')

为了挖掘出各个流派中的高IMDB评分作品演员，我们需要先根据流派和演员进行分组。

对演员进行分组的时候，选择的是用`person_id`而不是`name`变量，原因是名字容易出现错拼或者重名的情况，演职员ID会比演员姓名更加准确地反映是哪位演员。

In [ ]:
groupby_genres_and_person_id = actor_with_titles.groupby(["genres", "person_id"])

分组后，我们只需要对`imdb_score`的值进行聚合计算，因此只提取`imdb_score`变量，然后调用`mean`，来计算各个流派影视作品中，每位演员参演作品的平均IMDB评分。

In [ ]:
imdb_score_groupby_genres_and_person_id = groupby_genres_and_person_id["imdb_score"].mean()
imdb_score_groupby_genres_and_person_id

我们可以调用`reset_index`，对层次化索引进行重置，得到更加规整的DataFrame。

In [ ]:
imdb_score_groupby_genres_and_person_id_df = imdb_score_groupby_genres_and_person_id.reset_index()
imdb_score_groupby_genres_and_person_id_df

现在针对流派和演员分组的IMDB评分数据已经整理好，可以进入后续的分析步骤了。

但我们当前可以继续做一些数据整理，比如对上面的结果再次进行分组，找出各个流派里演员作品最高的平均评分是多少、最高评分对应的演员名字是什么。

要得到这一结果，我们需要再次用`genres`进行分组，然后提取出`imdb_score`变量，计算其最大值。

In [ ]:
genres_max_scores = imdb_score_groupby_genres_and_person_id_df.groupby("genres")["imdb_score"].max()
genres_max_scores

在我们知道最高分后，可以把以上结果和之前得到的`imdb_score_groupby_genres_and_person_id_df`再次进行连接，得到最高分对应的各个演员ID是什么，也就是这个最高平均分是哪位演员拿到的。

In [ ]:
genres_max_score_with_person_id = pd.merge(imdb_score_groupby_genres_and_person_id_df, genres_max_scores, on=["genres", "imdb_score"])
genres_max_score_with_person_id

从以上结果可以看出，最高分对应的演员不一定只有一位，可能有多位演员的平均得分相同。

为了得到演员ID所对应的演员名字，我们可以和`cleaned_credits`这个DataFrame进行连接。这个DataFrame还有其它列，我们只需要得到`person_id`和`name`的对应，所以可以先提取出那两列，并把重复行删除。

In [ ]:
actor_id_with_names = cleaned_credits[['person_id', 'name']].drop_duplicates()
actor_id_with_names.head(10)

下一步就可以把`actor_id_with_names`与前面得到的`genres_max_score_with_person_id`进行连接，增加`name`变量，从而展示平均评分最高的演员名字。

In [ ]:
genres_max_score_with_actor_name = pd.merge(genres_max_score_with_person_id, actor_id_with_names, on="person_id")
genres_max_score_with_actor_name

为了把相同流派都排序在一起，我们还可以用`sort_values`方法，把结果里面的行根据`genres`进行排序，然后用`reset_index`把索引重新排序。

索引重新排序后，DataFrame会多出`index`一列，我们可以再把`index`列进行删除。

In [ ]:
genres_max_score_with_actor_name = genres_max_score_with_actor_name.sort_values("genres").reset_index().drop("index", axis=1)
genres_max_score_with_actor_name